This notebook uses [mvncall](https://mathgen.stats.ox.ac.uk/genetics_software/mvncall/mvncall.html) to phase three multiallelic SNPs within VGSC.

In [2]:
%run setup.ipynb

## install mvncall
- mvncall depends on a couple of boost libraries, I installed these first using "sudo apt-get install libboost-dev"

In [3]:
%%bash --err install_err --out install_out

# This script downloads and installs mvncall. We won't include this in 
# the standard install.sh script as this is not something we want to do
# as part of continuous integration, it is only needed for this data 
# generation task.

set -xeo pipefail

mkdir -p ../dependencies

cd ../dependencies

if [ ! -f mvncall.installed ]; then
    echo installing mvncall
    
    # clean up
    rm -rvf mvncall*
    
    # download and unpack
    wget https://mathgen.stats.ox.ac.uk/genetics_software/mvncall/mvncall_v1.0_x86_64_dynamic.tgz
    tar zxvf mvncall_v1.0_x86_64_dynamic.tgz
    
    # trick mnvcall into finding boost libraries - their names aren't what mvncall expects
    locate libboost_iostreams | xargs -I '{}' ln -v -f -s '{}' libboost_iostreams.so.5
    locate libboost_program_options | xargs -I '{}' ln -v -f -s '{}' libboost_program_options.so.5
    
    # try running mvncall
    export LD_LIBRARY_PATH=.
    ./mvncall_v1.0_x86_64_dynamic/mvncall 
    
    # mark success
    touch mvncall.installed

else
    echo mvncall already installed
    
fi

In [4]:
#check install
print(install_out)

mvncall already installed



In [5]:
# check we can run mvncall
mvncall = 'LD_LIBRARY_PATH=../dependencies ../dependencies/mvncall_v1.0_x86_64_dynamic/mvncall'
!{mvncall}



  MVNcall version 1.0


Copyright 2013 Androniki Menelaou and Jonathan Marchini
Please see the LICENCE file included with this program for conditions of use.


INPUT FILES

    --sample-file <file>                A list of the sample IDs names 
    --glfs <file>                       Genotype likelihoods input file (vcf format)
    --scaffold-file <file>              Haplotype scaffold 


MODEL OPTIONS

    --int <lower> <upper>               Lower and upper bounds (physical positions) of the region of interest. 
    --list <file>                       A list of all positions to be tested [instead of interval] 
    --numsnps <int>                     Number of SNPs at either site of the position of interest [default 50]
    --lambda <double>                   Penalty value [default 0.06]
    --ped-file <file>                   File in .ped format
    --k <int>                           The number of haplotypes to be selected for inference, [default k = all ]
    --model-avg          

## prepare input files

In [6]:
# these are the source data files for the phasing
sample_file = '../phase2.AR1/haplotypes/main/shapeit/ag1000g.phase2.ar1.samples.2L.gz'
scaffold_file = '../phase2.AR1/haplotypes/main/shapeit/ag1000g.phase2.ar1.haplotypes.2L.gz'

## list of SNPs to phase

In [7]:
# this file will contain the list of SNPs to be phased
list_file = '../data/variants_mvncall_phase2.list'

In [8]:
%%file {list_file}
2391228
2400071
2431061

Overwriting ../data/variants_mvncall_phase2.list


In [9]:
# for mvncall we need a simple manifest of sample IDs
# N.B., we will exclude the cross parents
!gunzip -v {sample_file} -c | head -n 1144 | tail -n 1142 | cut -d' ' -f1 > /tmp/ag1000g.phase2.ar1.samples.2L

../phase2.AR1/haplotypes/main/shapeit/ag1000g.phase2.ar1.samples.2L.gz:	 77.0%


In [10]:
!head /tmp/ag1000g.phase2.ar1.samples.2L

AA0040-C
AA0041-C
AA0042-C
AA0043-C
AA0044-C
AA0048-C
AA0049-C
AA0050-C
AA0051-C
AA0052-C


In [11]:
!tail /tmp/ag1000g.phase2.ar1.samples.2L

AY0079-C
AY0080-C
AY0082-C
AY0083-C
AY0085-C
AY0087-C
AY0088-C
AY0089-C
AY0090-C
AY0091-C


In [12]:
!wc -l /tmp/ag1000g.phase2.ar1.samples.2L

1142 /tmp/ag1000g.phase2.ar1.samples.2L


## haplotype scaffold

In [13]:
# mvncall needs the haps unzipped. Also we will exclude the cross parents
!if [ ! -f /tmp/ag1000g.phase2.ar1.haplotypes.2L ]; then gunzip -v {scaffold_file} -c | cut -d' ' -f1-2289 > /tmp/ag1000g.phase2.ar1.haplotypes.2L; fi

../phase2.AR1/haplotypes/main/shapeit/ag1000g.phase2.ar1.haplotypes.2L.gz:	 98.8%


In [14]:
# check cut has worked
!head -n1 /tmp/ag1000g.phase2.ar1.haplotypes.2L

2 . 25050 G T 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [15]:
!tail -n1 /tmp/ag1000g.phase2.ar1.haplotypes.2L

2 . 49356429 A G 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0

In [16]:
# check cut has worked
!head -n1 /tmp/ag1000g.phase2.ar1.haplotypes.2L | wc

      1    2289    4582


In [17]:
# mvncall needs an unzipped VCF, we'll extract only the region we need
region_vgsc = SeqFeature('2L', 2358158, 2431617)
region_vgsc.region_str

'2L:2358158-2431617'

# up to here - awaiting vcf download

In [18]:
vcf_file = '/media/chris/Seagate\ Expansion\ Drive/kwiat/vector/ag1000g/release/phase2.AR1/variation/main/vcf/all/ag1000g.phase2.ar1.2L.vcf.gz'

In [19]:
# extract the VCF
!bcftools view -r {region_vgsc.region_str} --output-file /tmp/vgsc.vcf --output-type v {vcf_file}

In [20]:
%%bash

for numsnps in 50 100 200; do
    echo $numsnps
done

50
100
200


In [21]:
%%bash

# run mvncall, only if output file doesn't exist (it's slow)

mvncall="../dependencies/mvncall_v1.0_x86_64_dynamic/mvncall"
export LD_LIBRARY_PATH=../dependencies

for numsnps in 50 100 200; do
    output_file=../data/phasing_extra_phase2.mvncall.${numsnps}.vcf

    if [ ! -f $output_file ]; then 
        echo running mvncall $numsnps
        $mvncall \
            --sample-file /tmp/ag1000g.phase2.ar1.samples.2L \
            --glfs /tmp/vgsc.vcf \
            --scaffold-file /tmp/ag1000g.phase2.ar1.haplotypes.2L \
            --list ../data/variants_mvncall_phase2.list \
            --numsnps $numsnps \
            --o $output_file > /tmp/mvncall.${numsnps}.log 
    else
        echo skipping mvncall $numsnps
    fi
    
done

skipping mvncall 50
skipping mvncall 100
running mvncall 200


In [22]:
!tail /tmp/mvncall.200.log

Analysing variant 3/3

Writing phased genotypes in output files...

Time elapsed : 4175.713265

~~~ ΤΕΛΟΣ ~~~ 

Have a nice day!



In [128]:
!cat ../data/phasing_extra_phase2.mvncall.200.vcf

##fileformat=VCFv4.0
##source=MVNcall
##FORMAT=<ID=GT,Number=1,Type=String,Description='Genotype'>
##INFO=<ID=NUMALT,Number=1,Type=Integer,Description='Number of alternative alleles'>
##INFO=<ID=AVGPOST,Number=1,Type=Float,Description='Average posterior probability from MVNcall'>
##FORMAT=<ID=PL,Number=.,Type=Integer,Description='Phred-scaled Genotype Likelihoods'>
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	AA0040-C	AA0041-C	AA0042-C	AA0043-C	AA0044-C	AA0048-C	AA0049-C	AA0050-C	AA0051-C	AA0052-C	AA0053-C	AA0054-C	AA0055-C	AA0056-C	AA0060-C	AA0061-C	AA0063-C	AA0064-C	AA0066-C	AA0067-C	AA0068-C	AA0072-C	AA0073-C	AA0074-C	AA0075-C	AA0076-C	AA0077-C	AA0080-C	AA0084-C	AA0085-C	AA0086-C	AA0087-C	AA0088-C	AA0089-C	AA0090-C	AA0091-C	AA0096-C	AA0097-C	AA0098-C	AA0099-C	AA0100-C	AA0101-C	AA0102-C	AA0103-C	AA0104-C	AA0107-C	AA0108-C	AA0109-C	AA0110-C	AA0111-C	AA0113-C	AA0114-C	AA0115-C	AA0116-C	AA0122-C	AA0123-C	AA0124-C	AA0125-C	AA0127-C	AA0132-C	AA0133-C	AA0134-C	AA0135-C	AA0136-C	AA0139-C	A

In [129]:
!ls -lh ../data/*.mvncall*

-rw-rw-r-- 1 chris chris 1.2K Aug 16 18:13 ../data/phasing_extra_phase2.mvncall.100.npz
-rw-rw-r-- 1 chris chris  76K Aug 16 11:13 ../data/phasing_extra_phase2.mvncall.100.vcf
-rw-rw-r-- 1 chris chris 1.2K Aug 16 18:13 ../data/phasing_extra_phase2.mvncall.200.npz
-rw-rw-r-- 1 chris chris  76K Aug 16 14:16 ../data/phasing_extra_phase2.mvncall.200.vcf
-rw-rw-r-- 1 chris chris 1.2K Aug 16 18:13 ../data/phasing_extra_phase2.mvncall.50.npz
-rw-rw-r-- 1 chris chris  76K Aug 16 10:53 ../data/phasing_extra_phase2.mvncall.50.vcf


## convert to numpy arrays
- so we can interleave these variants back into the genotype array easily

In [145]:
def vcf_to_numpy(numsnps):
    
    # input VCF filename
    vcf_fn = '../data/phasing_extra_phase2.mvncall.{}.vcf'.format(numsnps)
    
    # extract
    allel.vcf_to_npz(vcf_fn, vcf_fn[:-3] + 'npz', overwrite=True)

In [146]:
for numsnps in 50, 100, 200:
    vcf_to_numpy(numsnps)

check parsing...

In [152]:
callset = np.load('test.npz', allow_pickle=True)

In [153]:
sorted(callset.keys())

['calldata/GT',
 'samples',
 'variants/ALT',
 'variants/CHROM',
 'variants/FILTER_PASS',
 'variants/ID',
 'variants/POS',
 'variants/QUAL',
 'variants/REF']

In [154]:
#lets check the genotypes
g = allel.GenotypeArray(callset['calldata/GT'])
#need bool of which geno are phased - all in this case
p_bool = np.ones((3,1142))
#can then roni size as a haplotype array
g.is_phased = p_bool
g.displayall()

<GenotypeArray shape=(3, 1142, 2) dtype=int8> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342 343 344 345 346 347 348 349 350 351 352 353 354 355 356 357 358 359 360 361 362 363 364 365 366 367 368 369 370 371 372 373 374 375 376 377 378 379 380 381 382 383 384 385 386 387 388 389 390 391 392 393 394 395 396 397 398 399 400 401 402 403 404 405 406 407 408 409 410 411 412 413 414 415 416 417 418 419 420 421 422 423 424 425 426 427 428 429 430 431 432 433 434 435 436 437 438 439 440 441 442 443 444 445 446 447 448 449 450 451 452 453 454 455 456 457 458 459 460 461 462 463 464 465 466 467 468 469 470 471 472 473 474 475 476 477 478 479 480 481 482 483 484 485 486 487 488 489 490 491 492 493 494 495 496 497 498 499 500 501 502 503 504 505 506 507 508 509 510 511 512 513 514 515 516 517 518 519 520 521 522 523 524 525 526 527 528 529 530 531 532 533 534 535 536 537 538 539 540 541 542 543 544 545 546 547 548 549 550 551 552 553 554 555 556 557 558 559 560 561 562 563 564 565 566 567 568 569 570 571 572 573 574 575 576 577 578 579 580 581 582 583 584 585 586 587 588 589 590 591 592 593 594 595 596 597 598 599 600 601 602 603 604 605 606 607 608 609 610 611 612 613 614 615 616 617 618 619 620 621 622 623 624 625 626 627 628 629 630 631 632 633 634 635 636 637 638 639 640 641 642 643 644 645 646 647 648 649 650 651 652 653 654 655 656 657 658 659 660 661 662 663 664 665 666 667 668 669 670 671 672 673 674 675 676 677 678 679 680 681 682 683 684 685 686 687 688 689 690 691 692 693 694 695 696 697 698 699 700 701 702 703 704 705 706 707 708 709 710 711 712 713 714 715 716 717 718 719 720 721 722 723 724 725 726 727 728 729 730 731 732 733 734 735 736 737 738 739 740 741 742 743 744 745 746 747 748 749 750 751 752 753 754 755 756 757 758 759 760 761 762 763 764 765 766 767 768 769 770 771 772 773 774 775 776 777 778 779 780 781 782 783 784 785 786 787 788 789 790 791 792 793 794 795 796 797 798 799 800 801 802 803 804 805 806 807 808 809 810 811 812 813 814 815 816 817 818 819 820 821 822 823 824 825 826 827 828 829 830 831 832 833 834 835 836 837 838 839 840 841 842 843 844 845 846 847 848 849 850 851 852 853 854 855 856 857 858 859 860 861 862 863 864 865 866 867 868 869 870 871 872 873 874 875 876 877 878 879 880 881 882 883 884 885 886 887 888 889 890 891 892 893 894 895 896 897 898 899 900 901 902 903 904 905 906 907 908 909 910 911 912 913 914 915 916 917 918 919 920 921 922 923 924 925 926 927 928 929 930 931 932 933 934 935 936 937 938 939 940 941 942 943 944 945 946 947 948 949 950 951 952 953 954 955 956 957 958 959 960 961 962 963 964 965 966 967 968 969 970 971 972 973 974 975 976 977 978 979 980 981 982 983 984 985 986 987 988 989 990 991 992 993 994 995 996 997 998 999 1000 1001 1002 1003 1004 1005 1006 1007 1008 1009 1010 1011 1012

In [155]:
callset['variants/ALT']

array([['C', 'T', ''],
       ['A', 'T', ''],
       ['A', 'T', '']], dtype=object)

In [158]:
callset['variants/REF']

array(['G', 'G', 'C'], dtype=object)

## can we easily turn the variants into a variant table?

In [164]:
list(callset.keys())

['samples',
 'calldata/GT',
 'variants/ALT',
 'variants/CHROM',
 'variants/FILTER_PASS',
 'variants/ID',
 'variants/POS',
 'variants/QUAL',
 'variants/REF']

In [8]:
allel.VariantTable(variants)

<VariantTable shape=(3,) dtype=(numpy.record, [('CHROM', 'S12'), ('POS', '<i4'), ('ID', 'S12'), ('REF', 'S1'), ('ALT', 'S1'), ('QUAL', '<f4'), ('FILTER_PASS', '?'), ('num_alleles', 'u1'), ('is_snp', '?'), ('svlen', '<i4'), ('AVGPOST', '<f4'), ('NUMALT', '<i4')])>
[(b'2L', 2391228, b'.', b'G', b'C', 0.0, True, 3, True, 0, 0.0, 2)
 (b'2L', 2400071, b'.', b'G', b'A', 0.0, True, 3, True, 0, 0.0, 2)
 (b'2L', 2429745, b'.', b'A', b'T', 0.0, True, 2, True, 0, 0.0, 1)]

In [9]:
calldata = callset['calldata']

In [10]:
g = allel.GenotypeArray(calldata['genotype'])
g.is_phased = calldata['is_phased']
g.displayall()

<GenotypeArray shape=(3, 765, 2) dtype=int8> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342 343 344 345 346 347 348 349 350 351 352 353 354 355 356 357 358 359 360 361 362 363 364 365 366 367 368 369 370 371 372 373 374 375 376 377 378 379 380 381 382 383 384 385 386 387 388 389 390 391 392 393 394 395 396 397 398 399 400 401 402 403 404 405 406 407 408 409 410 411 412 413 414 415 416 417 418 419 420 421 422 423 424 425 426 427 428 429 430 431 432 433 434 435 436 437 438 439 440 441 442 443 444 445 446 447 448 449 450 451 452 453 454 455 456 457 458 459 460 461 462 463 464 465 466 467 468 469 470 471 472 473 474 475 476 477 478 479 480 481 482 483 484 485 486 487 488 489 490 491 492 493 494 495 496 497 498 499 500 501 502 503 504 505 506 507 508 509 510 511 512 513 514 515 516 517 518 519 520 521 522 523 524 525 526 527 528 529 530 531 532 533 534 535 536 537 538 539 540 541 542 543 544 545 546 547 548 549 550 551 552 553 554 555 556 557 558 559 560 561 562 563 564 565 566 567 568 569 570 571 572 573 574 575 576 577 578 579 580 581 582 583 584 585 586 587 588 589 590 591 592 593 594 595 596 597 598 599 600 601 602 603 604 605 606 607 608 609 610 611 612 613 614 615 616 617 618 619 620 621 622 623 624 625 626 627 628 629 630 631 632 633 634 635 636 637 638 639 640 641 642 643 644 645 646 647 648 649 650 651 652 653 654 655 656 657 658 659 660 661 662 663 664 665 666 667 668 669 670 671 672 673 674 675 676 677 678 679 680 681 682 683 684 685 686 687 688 689 690 691 692 693 694 695 696 697 698 699 700 701 702 703 704 705 706 707 708 709 710 711 712 713 714 715 716 717 718 719 720 721 722 723 724 725 726 727 728 729 730 731 732 733 734 735 736 737 738 739 740 741 742 743 744 745 746 747 748 749 750 751 752 753 754 755 756 757 758 759 760 761 762 763 764 0 0|0 2|0 0|0 0|0 0|2 0|2 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 1|0 0|0 1|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|1 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|1 0|0 0|0 0|0 0|0 0|0 0|0 0|2 0|0 0|0 0|0 0|2 0|0 0|0 0|0 0|0 0|0 0|0 1|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 2|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 1|0 0|0 0|0 2|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 1|0 0|0 0|0 0|0 0|0 0|1 0|0 0|0 0|0 0|0 0|0 0|0 0|2 0|0 0|0 0|0 0|0 1|0 0|0 0|0 2|0 0|1 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0 0|0

In [ ]:
#can we turn the variants w